In [ ]:
# /usr/local/bin/python3.12 -m pip install "tensorflow==2.20.*"

# Full optimisation


In [ ]:


import os, sys, warnings, subprocess
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from typing import List

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, roc_auc_score, average_precision_score

# ---------------- CONFIG ----------------
RANDOM_STATE     = 42
VIBE_HIGH_TAU    = 0.0375           # label split for VIBE1
RECALL_FLOOR     = 0.60             # recall floor for positives (high-VIBE)
FEATURES_5       = ['GRAVY_VH','SAP_pos_Fv','GRAVY_HCDR3','SAP_pos_Hv','SAP_pos_Lv']
FEATURES_38      = [
    'SAP_pos_CDRH1','SAP_pos_CDRH2','SAP_pos_CDRH3','SAP_pos_CDRL1','SAP_pos_CDRL2','SAP_pos_CDRL3',
    'SAP_pos_CDR','SAP_pos_Hv','SAP_pos_Lv','SAP_pos_Fv',
    'SCM_neg_CDRH1','SCM_neg_CDRH2','SCM_neg_CDRH3','SCM_neg_CDRL1','SCM_neg_CDRL2','SCM_neg_CDRL3',
    'SCM_neg_CDR','SCM_neg_Hv','SCM_neg_Lv','SCM_neg_Fv',
    'SCM_pos_CDRH1','SCM_pos_CDRH2','SCM_pos_CDRH3','SCM_pos_CDRL1','SCM_pos_CDRL2','SCM_pos_CDRL3',
    'SCM_pos_CDR','SCM_pos_Hv','SCM_pos_Lv','SCM_pos_Fv',
    'GRAVY_VH','GRAVY_VL','GRAVY_HCDR1','GRAVY_HCDR2','GRAVY_HCDR3',
    'GRAVY_LCDR1','GRAVY_LCDR2','GRAVY_LCDR3'
]

DATA_DESCR       = Path("final_all_descriptors.csv")
DEEPSP_IN        = Path("DeepSP_input.csv")
DEEPSP_OUT       = Path("DeepSP_descriptors.csv")  
DEEPPRED_SCRIPT  = Path(os.environ.get("DEEPSP_SCRIPT", "deepsp_predictor.py"))
OUTDIR           = Path("opt_outputs_desc"); OUTDIR.mkdir(exist_ok=True, parents=True)

# Mutagenesis limits 
MAX_HCDR3_POS = 6
MAX_LCDR3_POS = 0           
MUTS_PER_POS  = 2

# Descriptor-only hydrophobicity score weights
HYDRO_WEIGHT  = {'SAP_pos_Fv':1.0,'SAP_pos_Hv':1.0,'SAP_pos_Lv':1.0,'GRAVY_VH':0.5,'GRAVY_HCDR3':0.5}

# helpers
def ensure_cols(df: pd.DataFrame, cols: List[str]):
    miss = [c for c in cols if c not in df.columns]
    if miss: raise ValueError(f"Missing columns: {miss}")

def kyte_doolittle(seq: str) -> float:
    kd = {'I':4.5,'V':4.2,'L':3.8,'F':2.8,'C':2.5,'M':1.9,'A':1.8,'G':-0.4,'T':-0.7,
          'S':-0.8,'W':-0.9,'Y':-1.3,'P':-1.6,'H':-3.2,'E':-3.5,'Q':-3.5,'D':-3.5,'N':-3.5,'K':-3.9,'R':-4.5}
    vals = [kd.get(a,0) for a in seq]
    return float(np.mean(vals)) if vals else 0.0

def choose_threshold_with_recall_floor(y_true, proba, recall_floor=RECALL_FLOOR) -> float:
    prec, rec, thr = precision_recall_curve(y_true, proba)
    f1 = 2*(prec*rec)/np.maximum(prec+rec,1e-12)
    mask = rec >= recall_floor
    if mask.any():
        k = np.argmax(f1[mask]); idx = np.arange(len(f1))[mask][k]
        return float(thr[max(idx-1,0)] if idx>0 else 0.0)
    idx = np.argmax(f1); return float(thr[max(idx-1,0)] if idx>0 else 0.0)

def zscore_from_ref(s: pd.Series, ref_mean: float, ref_std: float) -> pd.Series:
    sd = ref_std if ref_std != 0 else 1.0
    return (s - ref_mean) / sd

def creates_nglyc(seq: str) -> bool:
    # scan for N-X-[S/T] (X != P)
    for i in range(len(seq)-2):
        if seq[i]=='N' and seq[i+1] != 'P' and seq[i+2] in ('S','T'):
            return True
    return False

# load base table 
print("Loading final_all_descriptors.csv ...")
df = pd.read_csv(DATA_DESCR)
ensure_cols(df, ['Name','Status','VH','VL','VIBE1'] + FEATURES_38)

# Reference z-stats on the 234 set
z_means = df[FEATURES_38].mean()
z_stds  = df[FEATURES_38].std(ddof=0).replace(0, 1.0)

# train VIBE classifier on 5 features 
print("Training Elastic-Net VIBE classifier on 5 features ...")
df_v = df.dropna(subset=['VIBE1']).copy()
df_v['y'] = (df_v['VIBE1'] > VIBE_HIGH_TAU).astype(int)

X = df_v[FEATURES_5].values
y = df_v['y'].values

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y)

vibe_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        penalty="elasticnet", solver="saga", l1_ratio=0.2,
        C=0.1, class_weight="balanced", max_iter=5000, random_state=RANDOM_STATE
    ))
])

vibe_pipe.fit(X_tr, y_tr)
proba_te = vibe_pipe.predict_proba(X_te)[:,1]
thr_vibe = choose_threshold_with_recall_floor(y_te, proba_te, RECALL_FLOOR)
print(f"VIBE gate: ROC-AUC={roc_auc_score(y_te, proba_te):.3f} | AP={average_precision_score(y_te, proba_te):.3f} | thr={thr_vibe:.3f}")

#  generate single mutants (H/L CDR3)
print("Generating single mutants in CDR3 (heavy & light) ...")

HYDROPHOBIC = set("VILFMYWA")
POSITIVE    = set("KRH")
SUBS_FOR = {
    'V':['S','T'],'I':['S','T'],'L':['S','T'],'F':['S','N'],'M':['S','T'],'Y':['S','T'],'W':['S','N'],'A':['S','T'],
    'K':['Q','E'],'R':['Q','E'],'H':['Q','N'],
}

parents = df[df['Status'].str.lower().eq('terminated')][['Name','VH','VL']].copy()
mutants = []

for _, r in parents.iterrows():
    name, VH, VL = r['Name'], r['VH'], r['VL']
    # crude HCDR3 window: C...W 
    c = VH.find('C'); w = VH.rfind('W')
    if c != -1 and w != -1 and w > c:
        H_idx = list(range(c, min(w+1, len(VH))))
    else:
        H_idx = list(range(len(VH)//2, len(VH)))  # fallback: second half of VH
    # positions eligible for mutation
    cand_idx = [i for i in H_idx if i < len(VH) and (VH[i] in HYDROPHOBIC or VH[i] in POSITIVE)]
    cand_idx = cand_idx[:MAX_HCDR3_POS]

    for i in cand_idx:
        wt = VH[i]
        for m in SUBS_FOR.get(wt, [])[:MUTS_PER_POS]:
            if m == 'C': 
                continue
            newVH = VH[:i] + m + VH[i+1:]
            # avoid introducing NXS/T motif anywhere (conservative)
            if creates_nglyc(newVH) or creates_nglyc(VL):
                continue
            mutants.append((f"{name}|H{wt}{i+1}{m}", name, newVH, VL, 'H', i+1, wt, m))

# LCDR3 disabled 

mut_df = pd.DataFrame(mutants, columns=["MutantName","Parent","VH_mut","VL_mut","Chain","Position","WT","Mut"])
print(f"Mutants generated: {len(mut_df)} over {len(parents)} parents")
if mut_df.empty:
    print("No mutants produced — consider relaxing limits.")
    sys.exit(0)

# DeepSP recompute
print("Preparing DeepSP input and invoking deepsp_predictor.py ...")
DEEPSP_IN.unlink(missing_ok=True)
DEEPSP_OUT.unlink(missing_ok=True)

inp = pd.DataFrame({
    "Name": mut_df["MutantName"],
    "Heavy_Chain": mut_df["VH_mut"],
    "Light_Chain": mut_df["VL_mut"],
})
inp.to_csv(DEEPSP_IN, index=False)

# force predictor to run with the Python that has TF 2.20 + ANARCI 
DEEPSP_PYTHON = os.environ.get("DEEPSP_PYTHON", sys.executable)
print("DeepSP will run with:", DEEPSP_PYTHON)
env = os.environ.copy()
env["PATH"] = f"{Path(DEEPSP_PYTHON).parent}:{env.get('PATH','')}"
env["TF_USE_LEGACY_KERAS"] = "1"  

if not DEEPPRED_SCRIPT.exists():
    raise FileNotFoundError(f"DeepSP predictor not found at: {DEEPPRED_SCRIPT.resolve()} "
                            f"(set $DEEPSP_SCRIPT to override)")

res = subprocess.run([DEEPSP_PYTHON, str(DEEPPRED_SCRIPT)],
                     capture_output=True, text=True, env=env)
if res.returncode != 0:
    print("=== DeepSP STDOUT ===\n", res.stdout)
    print("=== DeepSP STDERR ===\n", res.stderr)
    raise RuntimeError(
        "DeepSP predictor failed. Ensure this Python has TensorFlow 2.20 and ANARCI.\n"
        "Tips:\n"
        "  conda activate deepsp\n"
        "  pip uninstall -y keras  # avoid Keras-3 conflicts\n"
        "  export DEEPSP_PYTHON=$(python -c 'import sys; print(sys.executable)')\n"
    )

if not DEEPSP_OUT.exists():
    raise FileNotFoundError("DeepSP_descriptors.csv was not produced by the predictor.")

# load DeepSP outputs & build feature table
deep = pd.read_csv(DEEPSP_OUT)
need = ['Name',
    'SAP_pos_CDRH1','SAP_pos_CDRH2','SAP_pos_CDRH3','SAP_pos_CDRL1','SAP_pos_CDRL2','SAP_pos_CDRL3',
    'SAP_pos_CDR','SAP_pos_Hv','SAP_pos_Lv','SAP_pos_Fv',
    'SCM_neg_CDRH1','SCM_neg_CDRH2','SCM_neg_CDRH3','SCM_neg_CDRL1','SCM_neg_CDRL2','SCM_neg_CDRL3',
    'SCM_neg_CDR','SCM_neg_Hv','SCM_neg_Lv','SCM_neg_Fv',
    'SCM_pos_CDRH1','SCM_pos_CDRH2','SCM_pos_CDRH3','SCM_pos_CDRL1','SCM_pos_CDRL2','SCM_pos_CDRL3',
    'SCM_pos_CDR','SCM_pos_Hv','SCM_pos_Lv','SCM_pos_Fv'
]
missing = [c for c in need if c not in deep.columns]
if missing:
    raise ValueError(f"DeepSP_descriptors.csv missing columns: {missing}")

# Sanity - ensure the outputs correspond exactly to mutants 
deep_names = set(deep['Name'].tolist())
mut_names  = set(mut_df['MutantName'].tolist())
if deep_names != mut_names:
    extra = deep_names - mut_names
    missingN = mut_names - deep_names
    raise ValueError(f"DeepSP name mismatch.\nExtra in DeepSP: {list(extra)[:5]}\nMissing in DeepSP: {list(missingN)[:5]}")

# compute GRAVY and build 38 features
name2vh = dict(zip(mut_df['MutantName'], mut_df['VH_mut']))
name2vl = dict(zip(mut_df['MutantName'], mut_df['VL_mut']))

gravy_rows = []
for n in deep['Name']:
    vh = name2vh[n]; vl = name2vl[n]
    gravy_vh   = kyte_doolittle(vh)
    gravy_hcdr3 = gravy_vh   
    gravy_vl   = kyte_doolittle(vl)
    gravy_rows.append((n, gravy_vh, gravy_hcdr3, gravy_vl))

gravy_df = pd.DataFrame(gravy_rows, columns=['Name','GRAVY_VH','GRAVY_HCDR3','GRAVY_VL'])
mut_desc = deep.merge(gravy_df, on='Name', how='left')

# fill remaining GRAVY_* CDRs with 0.0 - column completeness
for c in ['GRAVY_HCDR1','GRAVY_HCDR2','GRAVY_LCDR1','GRAVY_LCDR2','GRAVY_LCDR3']:
    mut_desc[c] = 0.0

# Build 5 features for VIBE gate
ensure_cols(mut_desc, FEATURES_5)
X_mut5 = mut_desc[FEATURES_5].values
p_high = vibe_pipe.predict_proba(X_mut5)[:,1]
mut_desc['VIBE_high_prob'] = p_high
mut_desc['VIBE_pred'] = (p_high >= thr_vibe).astype(int)  # 1 = high (reject)

# Attach meta 
mut_desc = mut_desc.merge(mut_df[['MutantName','Parent','Chain','Position','WT','Mut']],
                          left_on='Name', right_on='MutantName', how='left')

# rank by descriptor-only HydroScore 
survivors = mut_desc[mut_desc['VIBE_pred']==0].copy()
if survivors.empty:
    print("No mutants passed the VIBE gate. Consider raising the threshold or relaxing recall floor.")
    sys.exit(0)

# z-scores based on the 234-reference stats
for k in HYDRO_WEIGHT.keys():
    mu, sd = float(z_means[k]), float(z_stds[k])
    survivors[f"z_{k}"] = zscore_from_ref(survivors[k], mu, sd)

survivors["HydroScore38"] = sum(HYDRO_WEIGHT[k] * survivors[f"z_{k}"] for k in HYDRO_WEIGHT.keys())

# Final ranking: lowest VIBE prob, then lowest HydroScore38
ranked = survivors.sort_values(["VIBE_high_prob","HydroScore38"], ascending=[True, True]).copy()

# Save
OUTDIR.mkdir(exist_ok=True, parents=True)
ranked_cols = ['Parent','Name','Chain','Position','WT','Mut','VIBE_high_prob','HydroScore38'] + list(HYDRO_WEIGHT.keys())
ranked[ranked_cols].to_csv(OUTDIR/"optimized_mutants_descriptor_only.csv", index=False)

print(f"\nOptimization complete.")
print(f"Low-VIBE survivors ranked by HydroScore38 → {OUTDIR/'optimized_mutants_descriptor_only.csv'}")
print(f"Gate: thr={thr_vibe:.3f} (recall floor {RECALL_FLOOR:.2f})  |  survivors: {len(survivors)} / {len(mut_desc)}")


Loading final_all_descriptors.csv ...
Training Elastic-Net VIBE classifier on 5 features ...
VIBE gate: ROC-AUC=0.678 | AP=0.345 | thr=0.456
Generating single mutants in CDR3 (heavy & light) ...
Mutants generated: 1595 over 147 parents
Preparing DeepSP input and invoking deepsp_predictor.py ...
DeepSP will run with: /Users/tahuraazazattar/miniconda3/envs/deepsp_env/bin/python

Optimization complete.
Low-VIBE survivors ranked by HydroScore38 → opt_outputs_desc/optimized_mutants_descriptor_only.csv
Gate: thr=0.456 (recall floor 0.60)  |  survivors: 222 / 1595


# H3 optimisation

In [ ]:



import os, sys, warnings, subprocess, re
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple

# sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, roc_auc_score, average_precision_score

import matplotlib.pyplot as plt

RANDOM_STATE     = 42
VIBE_HIGH_TAU    = 0.0375           # label split for VIBE1  (1 = High-VIBE if VIBE1 > τ)
RECALL_FLOOR     = 0.60             # recall floor for positives (high-VIBE)
FEATURES_5       = ['GRAVY_VH','SAP_pos_Fv','GRAVY_HCDR3','SAP_pos_Hv','SAP_pos_Lv']
FEATURES_38      = [
    'SAP_pos_CDRH1','SAP_pos_CDRH2','SAP_pos_CDRH3','SAP_pos_CDRL1','SAP_pos_CDRL2','SAP_pos_CDRL3',
    'SAP_pos_CDR','SAP_pos_Hv','SAP_pos_Lv','SAP_pos_Fv',
    'SCM_neg_CDRH1','SCM_neg_CDRH2','SCM_neg_CDRH3','SCM_neg_CDRL1','SCM_neg_CDRL2','SCM_neg_CDRL3',
    'SCM_neg_CDR','SCM_neg_Hv','SCM_neg_Lv','SCM_neg_Fv',
    'SCM_pos_CDRH1','SCM_pos_CDRH2','SCM_pos_CDRH3','SCM_pos_CDRL1','SCM_pos_CDRL2','SCM_pos_CDRL3',
    'SCM_pos_CDR','SCM_pos_Hv','SCM_pos_Lv','SCM_pos_Fv',
    'GRAVY_VH','GRAVY_VL','GRAVY_HCDR1','GRAVY_HCDR2','GRAVY_HCDR3',
    'GRAVY_LCDR1','GRAVY_LCDR2','GRAVY_LCDR3'
]

DATA_DESCR       = Path("final_all_descriptors.csv")
DEEPSP_IN        = Path("DeepSP_input.csv")
DEEPSP_OUT       = Path("DeepSP_descriptors.csv")   # overwritten each run
DEEPPRED_SCRIPT  = Path(os.environ.get("DEEPSP_SCRIPT", "deepsp_predictor.py"))

OUTDIR           = Path("opt_outputs_H3"); OUTDIR.mkdir(exist_ok=True, parents=True)

# Mutagenesis limits 
MAX_HCDR3_POS = 6        # maximum positions in H3 to mutate per parent
MUTS_PER_POS  = 2        # maximum substitutions per chosen position

# Descriptor-only hydrophobicity score weights
HYDRO_WEIGHT  = {'SAP_pos_Fv':1.0,'SAP_pos_Hv':1.0,'SAP_pos_Lv':1.0,'GRAVY_VH':0.5,'GRAVY_HCDR3':0.5}

FAIL_STRICT = {"terminated","withdrawn","nfd","no further development"}

#  helpers
def ensure_cols(df: pd.DataFrame, cols: List[str]):
    miss = [c for c in cols if c not in df.columns]
    if miss: raise ValueError(f"Missing columns: {miss}")

def kyte_doolittle(seq: str) -> float:
    kd = {'I':4.5,'V':4.2,'L':3.8,'F':2.8,'C':2.5,'M':1.9,'A':1.8,'G':-0.4,'T':-0.7,
          'S':-0.8,'W':-0.9,'Y':-1.3,'P':-1.6,'H':-3.2,'E':-3.5,'Q':-3.5,'D':-3.5,'N':-3.5,'K':-3.9,'R':-4.5}
    vals = [kd.get(a,0) for a in seq]
    return float(np.mean(vals)) if vals else 0.0

def choose_threshold_with_recall_floor(y_true, proba, recall_floor=RECALL_FLOOR) -> float:
    prec, rec, thr = precision_recall_curve(y_true, proba)
    f1 = 2*(prec*rec)/np.maximum(prec+rec,1e-12)
    mask = rec >= recall_floor
    if mask.any():
        k = np.argmax(f1[mask]); idx = np.arange(len(f1))[mask][k]
        return float(thr[max(idx-1,0)] if idx>0 else 0.0)
    idx = np.argmax(f1); return float(thr[max(idx-1,0)] if idx>0 else 0.0)

def zscore_from_ref(s: pd.Series, ref_mean: float, ref_std: float) -> pd.Series:
    sd = ref_std if ref_std != 0 else 1.0
    return (s - ref_mean) / sd

def creates_nglyc(seq: str) -> bool:
    # conservative: scan whole sequence for N-X-[S/T] (X != P)
    for i in range(len(seq)-2):
        if seq[i]=='N' and seq[i+1] != 'P' and seq[i+2] in ('S','T'):
            return True
    return False

# H3 window: last C after FR3 (~80) to just before JH anchor WGQG/WGxG/FGxG/HGxG
def find_h3_window(vh: str) -> Tuple[int,int] | None:
    if not isinstance(vh, str) or len(vh) < 95: return None
    c_after80 = [m.start() for m in re.finditer('C', vh[80:])]
    if not c_after80: return None
    start = 80 + c_after80[-1]
    m = re.search(r'(W[AG]QG|WG.G|FG.G|HG.G)', vh[start:])
    if not m: return None
    stop = start + m.start()      # stop BEFORE anchor
    s, e = start+1, stop
    if e - s < 3: return None
    return (s, e)

#  load base table 
print("Loading final_all_descriptors.csv ...")
df = pd.read_csv(DATA_DESCR)
ensure_cols(df, ['Name','Status','VH','VL','VIBE1'] + FEATURES_38)

# Reference z-stats on the 234 set (for HydroScore scaling)
z_means = df[FEATURES_38].mean()
z_stds  = df[FEATURES_38].std(ddof=0).replace(0, 1.0)

#  train VIBE classifier on 5 features
print("Training Elastic-Net VIBE classifier on 5 features ...")
df_v = df.dropna(subset=['VIBE1']).copy()
df_v['y'] = (df_v['VIBE1'] > VIBE_HIGH_TAU).astype(int)       # 1 = High-VIBE

X = df_v[FEATURES_5].values
y = df_v['y'].values

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y)

vibe_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        penalty="elasticnet", solver="saga", l1_ratio=0.2,
        C=0.1, class_weight="balanced", max_iter=5000, random_state=RANDOM_STATE
    ))
])

vibe_pipe.fit(X_tr, y_tr)
proba_te = vibe_pipe.predict_proba(X_te)[:,1]
thr_vibe = choose_threshold_with_recall_floor(y_te, proba_te, RECALL_FLOOR)
print(f"VIBE gate: ROC-AUC={roc_auc_score(y_te, proba_te):.3f} | AP={average_precision_score(y_te, proba_te):.3f} | thr={thr_vibe:.3f}")

# generate single-site H3 mutants (heavy only)
print("Generating single-site HCDR3 mutants ...")

HYDROPHOBIC = set("VILFMYWA")
POSITIVE    = set("KRH")
SUBS_FOR = {
    'V':['S','T'],'I':['S','T'],'L':['S','T'],'F':['Y','S'],'M':['S','T'],'Y':['S','T'],'W':['Y','S'],'A':['S','T'],
    'K':['Q','E'],'R':['Q','E'],'H':['Q','N'],
}

parents = df[df['Status'].str.lower().isin(FAIL_STRICT)][['Name','VH','VL']].copy()
mutants = []

for _, r in parents.iterrows():
    name, VH, VL = r['Name'], r['VH'], r['VL']
    win = find_h3_window(VH)
    if not win: 
        continue
    s, e = win
    # candidate indices in H3 region (hydrophobic or + charged)
    cand_idx = [i for i in range(s, e) if VH[i] in HYDROPHOBIC or VH[i] in POSITIVE]
    cand_idx = cand_idx[:MAX_HCDR3_POS] if MAX_HCDR3_POS>0 else cand_idx

    for i in cand_idx:
        wt = VH[i]
        for m in SUBS_FOR.get(wt, [])[:MUTS_PER_POS]:
            if m == 'C': 
                continue
            newVH = VH[:i] + m + VH[i+1:]
            if creates_nglyc(newVH) or creates_nglyc(VL):
                continue
            mutants.append((f"{name}|H{i+1}{wt}{m}", name, newVH, VL, 'H', i+1, wt, m))

mut_df = pd.DataFrame(mutants, columns=["MutantName","Parent","VH_mut","VL_mut","Chain","Position","WT","Mut"])
print(f"Mutants generated: {len(mut_df)} over {len(parents)} parents")
if mut_df.empty:
    print("No mutants produced — consider relaxing limits.")
    sys.exit(0)

#  DeepSP: recompute for PARENTS + MUTANTS 
print("Preparing DeepSP input and invoking deepsp_predictor.py ...")
DEEPSP_IN.unlink(missing_ok=True)
DEEPSP_OUT.unlink(missing_ok=True)

inp_mut = pd.DataFrame({"Name": mut_df["MutantName"], "Heavy_Chain": mut_df["VH_mut"], "Light_Chain": mut_df["VL_mut"]})
inp_par = parents.rename(columns={"Name":"Name","VH":"Heavy_Chain","VL":"Light_Chain"})[["Name","Heavy_Chain","Light_Chain"]]
inp = pd.concat([inp_par, inp_mut], ignore_index=True).drop_duplicates(subset=["Name"])
inp.to_csv(DEEPSP_IN, index=False)

# force predictor to run with the Python that has TF 2.20 + ANARCI 
DEEPSP_PYTHON = os.environ.get("DEEPSP_PYTHON", sys.executable)
print("DeepSP will run with:", DEEPSP_PYTHON)
env = os.environ.copy()
env["PATH"] = f"{Path(DEEPSP_PYTHON).parent}:{env.get('PATH','')}"
env["TF_USE_LEGACY_KERAS"] = "1"

if not DEEPPRED_SCRIPT.exists():
    raise FileNotFoundError(f"DeepSP predictor not found at: {DEEPPRED_SCRIPT.resolve()} "
                            f"(set $DEEPSP_SCRIPT to override)")

res = subprocess.run([DEEPSP_PYTHON, str(DEEPPRED_SCRIPT)],
                     capture_output=True, text=True, env=env)
if res.returncode != 0:
    print("=== DeepSP STDOUT ===\n", res.stdout)
    print("=== DeepSP STDERR ===\n", res.stderr)
    raise RuntimeError("DeepSP predictor failed. Ensure TF2.20 + ANARCI are available.")

if not DEEPSP_OUT.exists():
    raise FileNotFoundError("DeepSP_descriptors.csv was not produced by the predictor.")

# load DeepSP outputs & assemble features
deep = pd.read_csv(DEEPSP_OUT)
need = ['Name',
    'SAP_pos_CDRH1','SAP_pos_CDRH2','SAP_pos_CDRH3','SAP_pos_CDRL1','SAP_pos_CDRL2','SAP_pos_CDRL3',
    'SAP_pos_CDR','SAP_pos_Hv','SAP_pos_Lv','SAP_pos_Fv',
    'SCM_neg_CDRH1','SCM_neg_CDRH2','SCM_neg_CDRH3','SCM_neg_CDRL1','SCM_neg_CDRL2','SCM_neg_CDRL3',
    'SCM_neg_CDR','SCM_neg_Hv','SCM_neg_Lv','SCM_neg_Fv',
    'SCM_pos_CDRH1','SCM_pos_CDRH2','SCM_pos_CDRH3','SCM_pos_CDRL1','SCM_pos_CDRL2','SCM_pos_CDRL3',
    'SCM_pos_CDR','SCM_pos_Hv','SCM_pos_Lv','SCM_pos_Fv'
]
missing = [c for c in need if c not in deep.columns]
if missing:
    raise ValueError(f"DeepSP_descriptors.csv missing columns: {missing}")

# SAP calibration: map DeepSP SAPs to original scale using parents 
parents_orig = df[['Name','SAP_pos_Fv','SAP_pos_Hv','SAP_pos_Lv']].dropna()
deep_par     = deep[['Name','SAP_pos_Fv','SAP_pos_Hv','SAP_pos_Lv']].dropna()
cal = parents_orig.merge(deep_par, on="Name", suffixes=("_orig","_deep"))

def fit_map(deep_vals, orig_vals):
    x = np.asarray(deep_vals, float)
    y = np.asarray(orig_vals, float)
    a, b = np.polyfit(x, y, 1)
    return float(a), float(b)

maps = {}
for key in ["SAP_pos_Fv","SAP_pos_Hv","SAP_pos_Lv"]:
    a,b = fit_map(cal[f"{key}_deep"], cal[f"{key}_orig"])
    maps[key] = (a,b)

def apply_calibration(df_in):
    out = df_in.copy()
    for key,(a,b) in maps.items():
        out[key] = a*out[key].astype(float) + b
    return out

deep_cal = apply_calibration(deep)

# build mutant feature table 
name2vh = dict(zip(mut_df['MutantName'], mut_df['VH_mut']))
name2vl = dict(zip(mut_df['MutantName'], mut_df['VL_mut']))

# GRAVY on VH and HCDR3 (heuristic window), plus GRAVY_VL 
gravy_rows = []
for n in mut_df['MutantName']:
    vh = name2vh[n]; vl = name2vl[n]
    gravy_vh   = kyte_doolittle(vh)
    # H3-only average using same window function
    win = find_h3_window(vh)
    if win:
        s,e = win; gravy_hcdr3 = kyte_doolittle(vh[s:e])
    else:
        gravy_hcdr3 = gravy_vh  # fallback
    gravy_vl   = kyte_doolittle(vl)
    gravy_rows.append((n, gravy_vh, gravy_hcdr3, gravy_vl))
gravy_df = pd.DataFrame(gravy_rows, columns=['Name','GRAVY_VH','GRAVY_HCDR3','GRAVY_VL'])

mut_desc = deep_cal[deep_cal['Name'].isin(mut_df['MutantName'])].merge(gravy_df, on='Name', how='left')
for c in ['GRAVY_HCDR1','GRAVY_HCDR2','GRAVY_LCDR1','GRAVY_LCDR2','GRAVY_LCDR3']:
    mut_desc[c] = 0.0

ensure_cols(mut_desc, FEATURES_5)

# Attach meta 
mut_desc = mut_desc.merge(mut_df[['MutantName','Parent','Chain','Position','WT','Mut']],
                          left_on='Name', right_on='MutantName', how='left')

# Attach parent baselines for Δ computations
parents_base = df[['Name','VIBE1','GRAVY_VH','GRAVY_HCDR3','SAP_pos_Fv','SAP_pos_Hv','SAP_pos_Lv','Status']].rename(
    columns={'Name':'Parent','VIBE1':'VIBE1_parent',
             'GRAVY_VH':'GRAVY_VH_parent','GRAVY_HCDR3':'GRAVY_HCDR3_parent',
             'SAP_pos_Fv':'SAP_pos_Fv_parent','SAP_pos_Hv':'SAP_pos_Hv_parent','SAP_pos_Lv':'SAP_pos_Lv_parent'})
mut_desc = mut_desc.merge(parents_base, on='Parent', how='left')

# VIBE gate on 5 features 
X_mut5 = mut_desc[FEATURES_5].values
p_high = vibe_pipe.predict_proba(X_mut5)[:,1]
mut_desc['VIBE_high_prob'] = p_high
mut_desc['VIBE_pred'] = (p_high >= thr_vibe).astype(int)  # 1 = high (reject)
mut_desc['p_low'] = 1.0 - mut_desc['VIBE_high_prob']

# Δ vs parent (after calibration) 
mut_desc['Delta_SAP_pos_Fv']  = mut_desc['SAP_pos_Fv']  - mut_desc['SAP_pos_Fv_parent']
mut_desc['Delta_SAP_pos_Hv']  = mut_desc['SAP_pos_Hv']  - mut_desc['SAP_pos_Hv_parent']
mut_desc['Delta_SAP_pos_Lv']  = mut_desc['SAP_pos_Lv']  - mut_desc['SAP_pos_Lv_parent']
mut_desc['Delta_GRAVY_VH']    = mut_desc['GRAVY_VH']    - mut_desc['GRAVY_VH_parent']
mut_desc['Delta_GRAVY_HCDR3'] = mut_desc['GRAVY_HCDR3'] - mut_desc['GRAVY_HCDR3_parent']

# survivors and ranking 
survivors = mut_desc[mut_desc['VIBE_pred']==0].copy()   # pass gate (low risk)
if survivors.empty:
    print("No mutants passed the VIBE gate. Consider raising the threshold or relaxing recall floor.")
else:
    # z-scores 
    for k in HYDRO_WEIGHT.keys():
        mu, sd = float(z_means[k]), float(z_stds[k])
        survivors[f"z_{k}"] = zscore_from_ref(survivors[k], mu, sd)
    survivors["HydroScore38"] = sum(HYDRO_WEIGHT[k] * survivors[f"z_{k}"] for k in HYDRO_WEIGHT.keys())

    ranked = survivors.sort_values(["VIBE_high_prob","HydroScore38"], ascending=[True, True]).copy()
    ranked_cols = ['Parent','Name','Chain','Position','WT','Mut','VIBE_high_prob','p_low','HydroScore38'] + list(HYDRO_WEIGHT.keys()) + \
                  ['Delta_SAP_pos_Fv','Delta_GRAVY_HCDR3']
    ranked[ranked_cols].to_csv(OUTDIR/"optimized_mutants_H3_ranked.csv", index=False)
    print(f"Survivors ranked by HydroScore38 → {OUTDIR/'optimized_mutants_H3_ranked.csv'}")

# Save full table
mut_desc.to_csv(OUTDIR/"optimized_mutants_H3_full.csv", index=False)
print(f"Full mutant table → {OUTDIR/'optimized_mutants_H3_full.csv'}")
print(f"Gate: thr={thr_vibe:.3f} (recall floor {RECALL_FLOOR:.2f})  |  survivors: {len(survivors)} / {len(mut_desc)}")

# PLOTS 
def savefig(path): 
    plt.tight_layout(); plt.savefig(path, dpi=220); plt.close()

# (1) Multi-objective landscape (ΔSAP_pos_Fv vs p_low)
plt.figure(figsize=(6.8,4.6))
plt.scatter(mut_desc['Delta_SAP_pos_Fv'], mut_desc['p_low'], alpha=0.75, edgecolor="black")
plt.axvline(0, linestyle="--", linewidth=1); plt.axhline(1.0-thr_vibe, linestyle="--", linewidth=1)
plt.xlabel("Δ SAP_pos_Fv (mutant − parent)  [smaller = better]")
plt.ylabel("P(Low-VIBE)  [bigger = better]")
plt.title("H3 optimisation — multi-objective landscape")
savefig(OUTDIR/"opt_H3_landscape.png")

# (2) Pareto front (non-dominated)
vals = mut_desc[['Delta_SAP_pos_Fv','p_low']].dropna().to_numpy()
dominated = np.zeros(len(vals), dtype=bool)
for i in range(len(vals)):
    better = (vals[:,0] <= vals[i,0]) & (vals[:,1] >= vals[i,1])
    strict = (vals[:,0] <  vals[i,0]) | (vals[:,1] >  vals[i,1])
    if np.any(better & strict): dominated[i] = True
pareto = mut_desc.loc[~dominated, ['Delta_SAP_pos_Fv','p_low']]

plt.figure(figsize=(6.8,4.6))
plt.scatter(mut_desc['Delta_SAP_pos_Fv'], mut_desc['p_low'], alpha=0.45, edgecolor="black", label="Mutants")
plt.scatter(pareto['Delta_SAP_pos_Fv'], pareto['p_low'], s=80, marker="D", edgecolor="black", label="Pareto")
plt.axvline(0, linestyle="--", linewidth=1); plt.axhline(1.0-thr_vibe, linestyle="--", linewidth=1)
plt.xlabel("Δ SAP_pos_Fv (smaller = better)"); plt.ylabel("P(Low-VIBE) (bigger = better)")
plt.title("H3 optimisation — Pareto front"); plt.legend()
savefig(OUTDIR/"opt_H3_pareto.png")

# (3) Funnel: total → pass gate → rescued (ΔSAP_pos_Fv<0)
resc_mask = (mut_desc['VIBE_pred']==0) & (mut_desc['Delta_SAP_pos_Fv'] < 0)
stages = ["Mutants","Pass gate","Rescued"]
counts = [len(mut_desc), int((mut_desc['VIBE_pred']==0).sum()), int(resc_mask.sum())]
plt.figure(figsize=(6.8,4.2))
plt.plot(range(len(stages)), counts, marker="o")
for i,(s,c) in enumerate(zip(stages, counts)): plt.text(i, c, str(c), ha="center", va="bottom")
plt.xticks(range(len(stages)), stages); plt.ylabel("Count"); plt.title("H3 optimisation — funnel")
savefig(OUTDIR/"opt_H3_funnel.png")

# (4) Top-15 rescued by ΔSAP_pos_Fv
resc = mut_desc[resc_mask].copy().sort_values("Delta_SAP_pos_Fv")
top15 = resc.head(15)
plt.figure(figsize=(8.2,4.8))
if not top15.empty:
    plt.barh(top15["Name"], -top15["Delta_SAP_pos_Fv"], edgecolor="black")
    plt.gca().invert_yaxis()
plt.xlabel("−Δ SAP_pos_Fv (bigger = more hydrophobicity reduction)")
plt.title("H3 optimisation — Top 15 rescued by ΔSAP_pos_Fv")
savefig(OUTDIR/"opt_H3_top15_rescued.png")

# (5) Slope chart: parent → best passing mutant (top 12 parents)
best = mut_desc[mut_desc['VIBE_pred']==0].copy()
if not best.empty:
    best = best.loc[best.groupby("Parent")["Delta_SAP_pos_Fv"].idxmin()].sort_values("Delta_SAP_pos_Fv").head(12)
    plt.figure(figsize=(8.2,4.8))
    for _, row in best.iterrows():
        plt.plot([0,1],[row["SAP_pos_Fv_parent"], row["SAP_pos_Fv"]], marker="o")
    plt.xticks([0,1], ["Parent","Best mutant (pass)"]); plt.ylabel("SAP_pos_Fv")
    plt.title("H3 optimisation — parent → best mutant (top 12)")
    savefig(OUTDIR/"opt_H3_slope_top12.png")

# (6) Δ-histograms (Pass vs Fail) for ΔGRAVY_HCDR3 and ΔSAP_pos_Fv
for col in ["Delta_GRAVY_HCDR3","Delta_SAP_pos_Fv"]:
    a = mut_desc.loc[mut_desc["VIBE_pred"]==0, col].dropna().values
    b = mut_desc.loc[mut_desc["VIBE_pred"]==1, col].dropna().values
    bins = np.histogram(np.concatenate([a,b]) if (len(a)+len(b))>0 else [0], bins=24)[1]
    plt.figure(figsize=(6.8,4.6))
    plt.hist(a, bins=bins, alpha=0.7, edgecolor="black", label="Pass")
    plt.hist(b, bins=bins, alpha=0.7, edgecolor="black", label="Fail")
    plt.axvline(0, linestyle="--", linewidth=1)
    plt.xlabel(f"{col}  (negative = improved)"); plt.ylabel("Count"); plt.legend()
    plt.title(f"H3 optimisation — {col} by gate")
    savefig(OUTDIR/f"opt_H3_hist_{col}.png")

# (7) Substitution matrix (WT → Mut)
subs = mut_desc.pivot_table(index="WT", columns="Mut", values="Name", aggfunc="count", fill_value=0)
plt.figure(figsize=(7.8,6.0))
plt.imshow(subs, aspect="auto")
plt.xticks(range(len(subs.columns)), subs.columns)
plt.yticks(range(len(subs.index)), subs.index)
plt.colorbar(label="Count"); plt.title("H3 optimisation — amino-acid substitutions (WT → Mut)")
savefig(OUTDIR/"opt_H3_substitution_matrix.png")

# (8) Positions targeted (H3)
pos_counts = mut_desc["Position"].value_counts().sort_index()
plt.figure(figsize=(7.8,4.0))
plt.bar(pos_counts.index.astype(str), pos_counts.values, edgecolor="black")
plt.xlabel("Heavy-chain position (HCDR3)"); plt.ylabel("Count")
plt.title("H3 optimisation — positions targeted")
savefig(OUTDIR/"opt_H3_positions.png")

# (9) Mini-radar gallery (4 parents): normalised per panel
def radar_one(row, out_png):
    feats = ["SAP_pos_Fv","SAP_pos_Hv","SAP_pos_Lv","GRAVY_VH","GRAVY_HCDR3"]
    pv = [row[f"{f}_parent"] for f in feats]
    mv = [row[f] for f in feats]
    allvals = np.array(pv+mv, float)
    mn, mx = np.nanmin(allvals), np.nanmax(allvals)
    if mx-mn == 0: 
        return
    pv = (np.array(pv)-mn)/(mx-mn); mv = (np.array(mv)-mn)/(mx-mn)
    ang = np.linspace(0, 2*np.pi, len(feats), endpoint=False)
    pv = np.concatenate([pv, pv[:1]]); mv = np.concatenate([mv, mv[:1]]); ang = np.concatenate([ang, ang[:1]])
    fig = plt.figure(figsize=(4.6,4.6)); ax = fig.add_subplot(111, polar=True)
    ax.plot(ang, pv); ax.fill(ang, pv, alpha=0.25)
    ax.plot(ang, mv); ax.fill(ang, mv, alpha=0.25)
    ax.set_xticks(ang[:-1]); ax.set_xticklabels(feats)
    ax.set_title(f"Biophysical profile: {row['Parent']}", y=1.08)
    fig.tight_layout(); fig.savefig(out_png, dpi=220); plt.close(fig)

passing = mut_desc[mut_desc['VIBE_pred']==0]
if not passing.empty:
    top4 = (passing.groupby("Parent")["Delta_SAP_pos_Fv"].min().sort_values().head(4)).index.tolist()
    for i,pn in enumerate(top4, 1):
        row = passing[passing["Parent"]==pn].sort_values("Delta_SAP_pos_Fv").iloc[0]
        radar_one(row, OUTDIR/f"opt_H3_radar_{i}.png")

print("Done. Outputs in:", OUTDIR.resolve())


Loading final_all_descriptors.csv ...
Training Elastic-Net VIBE classifier on 5 features ...
VIBE gate: ROC-AUC=0.678 | AP=0.345 | thr=0.456
Generating single-site HCDR3 mutants ...
Mutants generated: 1526 over 147 parents
Preparing DeepSP input and invoking deepsp_predictor.py ...
DeepSP will run with: /Users/tahuraazazattar/miniconda3/envs/deepsp_env/bin/python
Survivors ranked by HydroScore38 → opt_outputs_H3/optimized_mutants_H3_ranked.csv
Full mutant table → opt_outputs_H3/optimized_mutants_H3_full.csv
Gate: thr=0.456 (recall floor 0.60)  |  survivors: 983 / 1526
Done. Outputs in: /Users/tahuraazazattar/Library/CloudStorage/OneDrive-UniversityofBirmingham/Project/DeepSP/opt_outputs_H3
